In [21]:
import numpy as np
import pandas as pd

import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from src import nlp_utils
from src.process_text_variables import contracted_words_dict, stop_words_dict, punc, stop_words_incl_in_sentiment_dict 
from collections import Counter

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Functions

In [ ]:
### Bigrams

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 1) Import and Process Data

In [31]:
wsm = pd.read_csv('data/wallstreet_master.csv')
stop_words_dict = stop_words_dict + stop_words_incl_in_sentiment_dict
wsm['tokens'] = wsm['contentWithHTMLTag'].apply(lambda x: nlp_utils.process_text(x, contracted_words_dict, punc, stop_words_dict, min_len=2))

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'stop_words_incl_in_sentiment_dict' is not defined

In [4]:
all_docs_tokenized = wsm['tokens'].tolist()

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(all_docs_tokenized[:5])

## 2) Create Bigrams/Trigrams and Remove Neglible Words Based on Parts of Sentence (PoS)

In [11]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# English multi-task CNN trained on OntoNotes. Assigns context-specific token vectors, POS tags, dependency parse and named entities.
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADJ', 'PROPN']):
    texts_revised =[]
    for post in texts:
        doc = nlp(' '.join(post))
        texts_revised.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_revised

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Function Used to Calculate Score that Corresponds to "threshold" hyperparameter**

```
def original_scorer(worda_count, wordb_count, bigram_count, len_vocab, min_count, corpus_word_count):
   #...
   """
    worda_count : int
        Number of occurrences for first word.
    wordb_count : int
        Number of occurrences for second word.
    bigram_count : int
        Number of co-occurrences for phrase "worda_wordb".
    len_vocab : int
        Size of vocabulary.
    min_count: int
        Minimum collocation count threshold.
    corpus_word_count : int
        Not used in this particular scoring technique.
    """
    #...

    return (bigram_count - min_count) / worda_count / wordb_count * len_vocab
  ```

In [5]:
min_count = 10
# greater the threshold, the lower the number of words.
threshold = 50


bigram = gensim.models.Phrases(all_docs_tokenized, min_count=min_count, threshold=threshold)
trigram = gensim.models.Phrases(bigram[all_docs_tokenized], threshold=threshold)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
docs_incl_bigrams = make_bigrams(all_docs_tokenized)

docs_lemmatized = lemmatization(docs_incl_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADJ', 'PROPN'])

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
docs_lemmatized[22222]

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['understand',
 'right',
 'want',
 'move',
 'russia',
 'find',
 'teach',
 'hacker',
 'ask',
 'kinda',
 'joke',
 'day_change',
 'everythingcc_bank',
 'log',
 'room']

## 3) Create Corpus

In [24]:
# Bag of Words corpus
id2word = corpora.Dictionary(docs_lemmatized)

# Create unique ID for each word in corpus.
bow_corpus = [id2word.doc2bow(post) for post in  docs_lemmatized]

# Use the unique id as the index in id2word to see the corresponding token.
print(id2word[3])



C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


go


In [26]:
# Tfidf corpus
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
## 4) Develop LDA Model

In [27]:
# BoW 
lda_bow = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=3,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
# Tf-Idf
lda_tfidf = gensim.models.ldamodel.LdaModel(corpus=tfidf_corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=3,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
print('BoW TOPICS:')
for t in lda_bow.print_topics(): 
    print(t)
    print('\n')

BoW TOPICS:


C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'lda_bow' is not defined

In [40]:
print('Tf-Idf TOPICS:')
for t in lda_tfidf.print_topics(): 
    print(t)
    print('\n')

Tf-Idf TOPICS:
(0, '0.043*"escrow" + 0.031*"sign" + 0.022*"access" + 0.019*"release" + 0.018*"lol" + 0.016*"encrypt" + 0.015*"pas" + 0.012*"multiple" + 0.011*"admin" + 0.008*"document"')


(1, '0.103*"bank" + 0.094*"guide" + 0.032*"fullz" + 0.028*"interested" + 0.025*"usa" + 0.025*"cc" + 0.023*"paypal" + 0.022*"high_quality" + 0.021*"note" + 0.021*"login"')


(2, '0.034*"fresh" + 0.020*"bin" + 0.000*"bump" + 0.000*"_" + 0.000*"record_cashout" + 0.000*"phished_rule" + 0.000*"room" + 0.000*"everythingcc_bank" + 0.000*"day_change" + 0.000*"picsou"')


(3, '0.026*"verify" + 0.019*"worth" + 0.019*"save" + 0.018*"dream" + 0.016*"picture" + 0.014*"interesting" + 0.014*"advertise" + 0.012*"original" + 0.010*"choose" + 0.008*"photo"')


(4, '0.081*"drop" + 0.040*"way" + 0.034*"come" + 0.032*"cash" + 0.030*"friend" + 0.022*"score" + 0.018*"cheap" + 0.017*"good_luck" + 0.016*"call" + 0.015*"private"')


(5, '0.018*"mate" + 0.011*"cancel" + 0.003*"match" + 0.000*"bump" + 0.000*"_" + 0.000*"record_

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 4) Model Performance Evaluation

### Bag-of-Words

In [35]:
# The lower, the better.
print('Perplexity: ', lda_model.log_perplexity(bow_corpus))

coherence_model_bow = CoherenceModel(model=lda_model, texts=docs_lemmatized, dictionary=id2word, coherence='c_v')
coherence_bow = coherence_model_bow.get_coherence()
print('Coherence Score: ', coherence_bow)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity:  -17.050147888887906
Coherence Score:  0.37779212769099185


### Tf-Idf

In [41]:
print('Perplexity: ', lda_tfidf.log_perplexity(tfidf_corpus))

coherence_model_tfidf = CoherenceModel(model=lda_tfidf, texts=docs_lemmatized, dictionary=id2word, coherence='c_v')
coherence_tfidf = coherence_model_tfidf.get_coherence()
print('Coherence Score: ', coherence_tfidf)

C:\Users\morga\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity:  -20.00055618928151
Coherence Score:  0.3155524293093971
